In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import matplotlib.pyplot as plt
from datetime import datetime

In [13]:
conn = cx_Oracle.connect('ReportUser/report@10.10.1.7:1521/rproods')
cur = conn.cursor()

In [3]:
def getRecords(cur,sbs=100, store=None, dat='31/10/2019'):
    #dat = datetime.strptime(dat, '%d/%m/%Y')
    #dat  =  datetime.strftime("%d/%M/%Y",dat)
    
    query= "select i.INVC_SID, i.INVC_TYPE, i.CREATED_DATE,i.CREATED_HOUR, i.TENDER_TYPE, (t.QTY* t.PRICE) as EXT_PRICE, (t.QTY*t.COST) AS EXT_COST FROM INVC_ITEM_V t LEFT JOIN invoice_v i ON t.INVC_SID = i.INVC_SID where (i.SBS_NO={} AND i.STORE_NO={}) AND i.CREATED_DATE > to_date('{}','DD/MM/YYYY')".format(sbs, store, dat)
    
        
    cur.execute(query)
    records = cur.fetchall()
    columns = ['INVC_SID', 'INVC_TYPE', 'CREATED_DATE','CREATED_HOUR', 'TENDER_TYPE',  'EXT_PRICE',  'EXT_COST']
    df = pd.DataFrame(records, columns=columns)
    return df


In [21]:
def getRecordsforCount(cur,sbs=100, store=None, dat='31/10/2019'):
    #dat = datetime.strptime(dat, '%d/%m/%Y')
    #dat  =  datetime.strftime("%d/%M/%Y",dat)
    
    query= "select i.INVC_SID, i.INVC_TYPE, i.CREATED_DATE,i.CREATED_HOUR, i.TENDER_TYPE, (t.QTY* t.PRICE) as EXT_PRICE, (t.QTY*t.COST) AS EXT_COST FROM INVC_ITEM_V t LEFT JOIN invoice_v i ON t.INVC_SID = i.INVC_SID where (i.SBS_NO={} AND i.STORE_NO={} AND i.INVC_TYPE=0) AND i.CREATED_DATE > to_date('{}','DD/MM/YYYY')".format(sbs, store, dat)
    
        
    cur.execute(query)
    records = cur.fetchall()
    columns = ['INVC_SID', 'INVC_TYPE', 'CREATED_DATE','CREATED_HOUR', 'TENDER_TYPE',  'EXT_PRICE',  'EXT_COST']
    df = pd.DataFrame(records, columns=columns)
    return df


In [23]:
record = getRecordsforCount(cur, 100, 15)

In [25]:
hours  = [10,11,12,1,2,3,4,5,6]

In [27]:
record.head()

,INVC_SID,INVC_TYPE,CREATED_DATE,CREATED_HOUR,TENDER_TYPE,EXT_PRICE,EXT_COST
0,4405735294803198214,0,2019-10-31 07:38:55,7,2,3990,2603.0000
1,4405804592182679664,0,2019-10-31 08:15:09,8,2,6450,3850.0000
2,4405804592182679664,0,2019-10-31 08:15:09,8,2,2390,1435.0000
3,4405763917856291460,0,2019-10-31 08:18:17,8,0,715,423.0000
4,4405865850459249987,0,2019-10-31 08:19:10,8,0,70,35.5555


In [38]:
night_shift = record[(record.CREATED_HOUR.isin(hours))]

In [39]:
night_shift.head()

,INVC_SID,INVC_TYPE,CREATED_DATE,CREATED_HOUR,TENDER_TYPE,EXT_PRICE,EXT_COST
9,4406111013944255647,0,2019-10-31 10:05:17,10,2,50,33.4200
10,4406140967468095641,0,2019-10-31 10:37:51,10,2,3990,1780.0000
11,4406244019248425052,0,2019-10-31 10:51:37,10,2,1450,850.0000
12,4406244019248425052,0,2019-10-31 10:51:37,10,2,2200,1260.0000
13,4406260385512045302,0,2019-10-31 11:13:54,11,0,1750,1092.9189


In [69]:
night_shift['Month'] = night_shift.loc[night_shift.CREATED_DATE.dt.year]

KeyError: "None of [Int64Index([2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019,\n            ...\n            2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020],\n           dtype='int64', length=2949)] are in the [index]"

In [66]:
dt

9        10
10       10
11       10
12       10
13       10
         ..
15305     1
15306     1
15307     1
15308     1
15395     1
Name: CREATED_DATE, Length: 2949, dtype: int64

In [6]:
def PrepnSaveRecords(df, store):
    sales_values = k.groupby('INVC_SID').sum() #Sum all Invoice values
    sales = sales_values.drop(['CREATED_HOUR','TENDER_TYPE', 'INVC_TYPE'], axis = 1) # remove other columns so only 
    invc = df.drop_duplicates('INVC_SID').set_index('INVC_SID') # remove all duplicate invoices and set the index to invoice sid for merging
    s = pd.merge(sales, invc,  how='left', left_on = 'INVC_SID', right_on = 'INVC_SID' ) #Merge the sum of invoice withe original records
    s.drop(['EXT_PRICE_y', 'EXT_COST_y'], axis=1, inplace=True) #remove duplicate columns
    s.rename({'EXT_PRICE_x':'EXT_PRICE', 
             'EXT_COST_x':'EXT_COST'}, inplace = True)
    
    dta  = s.set_index('CREATED_DATE') #Make the record a time series 
  
    nm = "{}.csv".format(store)
    p  = dta.EXT_PRICE_x.resample('H').sum()
    d = pd.DataFrame(p, columns=['DATE','SALES'])
    p.to_csv(nm, header=True)
    return 


In [ ]:
def transcount(df, store):
    sales_values = k.groupby('INVC_SID').count() #Sum all Invoice values
    sales = sales_values.drop(['CREATED_HOUR','TENDER_TYPE', 'INVC_TYPE'], axis = 1) # remove other columns so only 
    invc = df.drop_duplicates('INVC_SID').set_index('INVC_SID') # remove all duplicate invoices and set the index to invoice sid for merging
    s = pd.merge(sales, invc,  how='left', left_on = 'INVC_SID', right_on = 'INVC_SID' ) #Merge the sum of invoice withe original records
    s.drop(['EXT_PRICE_y', 'EXT_COST_y'], axis=1, inplace=True) #remove duplicate columns
    s.rename({'EXT_PRICE_x':'EXT_PRICE', 
             'EXT_COST_x':'EXT_COST'}, inplace = True)
    
    dta  = s.set_index('CREATED_DATE') #Make the record a time series 
  
    nm = "{}.csv".format(store)
    p  = dta.EXT_PRICE_x.resample('H').sum()
    d = pd.DataFrame(p, columns=['DATE','SALES'])
    p.to_csv(nm, header=True)
    return 

In [7]:
stores = {11:'HP_ALLEN',15:'HP_LEKKI'}
dte = '31/10/2019'
def save_records(stores, dte):
    for s,v in stores.items(): 
        
        k = getRecords(cur, 100, s, dte)
        #print(v)
        #PrepnSaveRecords(k,v)
        PrepnSaveRecords(k,'HP_Lekki_18_11_2019')
        #print(s,v)
    return 


In [8]:
save_records(stores, dte)

NameError: name 'k' is not defined

In [10]:
# Run get function to generate the records from db. 
k = getRecords(cur, 100, 15, '31/10/2019') 
#Save the records to CSV
rec = PrepnSaveRecords(k,'HP_Lekki')